# LLM-Symbolic Explanation Refinement

Given a premise sentence $p_i$, a hypothesis sentence $h_i$, and an explanation $E_i$ consisting of a set of facts $\{f_1,f_2,...,f_n\}$, the explanation $E_i$ is considered logically valid if and only if the entailment $p_i \cup E_i \models h_i$ holds. This entailment is considered verifiable if $\{p_i, E_i, h_i\}$ can be translated into a set of logical formulae $\Phi$ that compose a theory $\Theta$. The validity of the theory $\Theta$ is subsequently determined by a theorem prover, verifying whether $\Theta \vDash \psi$, where $\psi$ represents a logical consequence derived from the logical form of $h_i$.

<img src="figures/nesy_refinement.png">

<img src="figures/nesy_refinement_results.png">

## Explanation Refinement (e-SNLI)

Premise: A couple playing with a little boy on the beach.

Hypothesis: "A couple are playing with a young child outside.

Annotated Explanation: little boy is a young child.

In [1]:
import nest_asyncio

nest_asyncio.apply()

# this also required by jupyter to start Isabelle server
import os

original_path = os.environ.get('PATH', '')
new_path = original_path + ':/Users/xinquan/Desktop/Isabelle2023.app/bin'
os.environ['PATH'] = new_path
print(os.environ['PATH'])

/Users/xinquan/miniforge3/envs/faithful/bin:/Users/xinquan/miniforge3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Users/xinquan/Desktop/Isabelle2023.app/bin


In [2]:
from refinement.refinement_model import RefinementModel
from generation.gpt import GPT
from critique import IsabelleCritique
import yaml

# data name is default to example_data
# premise and explanation are default to None
data_name = 'esnli_0'
premise = "A couple playing with a little boy on the beach."
hypothesis = "A couple are playing with a young child outside."
explanation = "little boy is a young child."

with open('config.yaml', 'r') as file:
     config = yaml.safe_load(file)
     api_key = config.get('gpt-4o', {}).get('api_key')

llm = GPT('gpt-4o', api_key)

# hard critique model
isabelle_solver = IsabelleCritique(
    generative_model=llm,
    isabelle_session = 'HOL',
    theory_name = data_name
)
prompt_dict = {
    'generate explanation': 'get_explanation_prompt.txt',
    'refine no premise': 'refine_hard_no_premise_prompt.txt',
    'refine with premise': 'refine_hard_with_premise_prompt.txt'
}

refinement_model = RefinementModel(
    generative_model=llm,
    critique_model=isabelle_solver,
    prompt_dict=prompt_dict
)

results = refinement_model.refine(
    hypothesis = hypothesis,
    premise = premise,
    explanation = explanation,
    data_name = data_name,
    iterations = 2
)

isabelle_solver.shutdown()

ValidationError: 4 validation errors for SessionBuildErrorResponse
response_body.ok
  Field required [type=missing, input_value={'kind': 'error', 'messag...: [], "verbose": true}'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
response_body.return_code
  Field required [type=missing, input_value={'kind': 'error', 'messag...: [], "verbose": true}'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
response_body.sessions
  Field required [type=missing, input_value={'kind': 'error', 'messag...: [], "verbose": true}'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
response_body.task
  Field required [type=missing, input_value={'kind': 'error', 'messag...: [], "verbose": true}'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [4]:
print("hypothesis:", results["hypothesis"])
print("premise:", results["premise"])
print("\n------------------ Explanation ----------------------\n")
print(results["refined explanation"])

hypothesis: A couple are playing with a young child outside.
premise: A couple playing with a little boy on the beach.

------------------ Explanation ----------------------

A little boy is a young child.
Being on the beach implies being outside.


In [17]:
import ast

iteration1_str = results["history critique output"][1]

start_index = iteration1_str.find('{')
if start_index == -1:
    raise ValueError("can't find start index")

dict_str = iteration1_str[start_index:]

iteration1_dict = ast.literal_eval(dict_str)

proof_tactics = iteration1_dict['proof tactics']
print(proof_tactics)


['Sledgehammering...', 'cvc4 found a proof...', 'zipperposition found a proof...', 'cvc4: Try this: using assms explanation_1 explanation_2 by blast (6 ms)', 'zipperposition: Found duplicate proof', 'cvc4 found a proof...', 'vampire found a proof...', 'cvc4 found a proof...', 'e found a proof...', 'spass found a proof...', 'cvc4: Found duplicate proof', 'zipperposition found a proof...', 'cvc4: Found duplicate proof', 'e: Found duplicate proof', 'zipperposition: Found duplicate proof', 'spass: Found duplicate proof', 'vampire: Found duplicate proof', 'zipperposition found a proof...', 'zipperposition: Found duplicate proof', 'Done']
